In [4]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"
from plotly.subplots import make_subplots
import folium 
from folium import plugins
from tqdm.notebook import tqdm as tqdm


import warnings
warnings.filterwarnings('ignore')

In [5]:
clean_data = pd.read_excel("DistrictWiseList324-converted.xlsx")

In [8]:
clean_data.dropna(subset=['Name of District'], inplace=True)

In [26]:
clean_data = clean_data.fillna(method='ffill')

In [31]:
import requests
import json

def get_geo_coordinate(location_text):
    try:
        resp = requests.get('https://www.geocode.xyz/'+location_text+"?json=1")
        resp_json = json.loads(resp.text)
        return resp_json['latt'], resp_json['longt']
    except:
        return '', ''

In [32]:
get_geo_coordinate('Delhi')

('28.65553', '77.23165')

In [33]:
clean_data['location_text'] = clean_data['Name of District'] + ", " + clean_data['State']

In [34]:
clean_data["Lat"], clean_data['Long'] = zip(*clean_data['location_text'].map(get_geo_coordinate))

In [35]:
clean_data.head()

,State,No of Districts Affected,Name of District,No of Cases,location_text,Lat,Long
0,Andhra Pradesh,5,East Godavari,2,"East Godavari, Andhra Pradesh",16.94374,82.23506
1,Andhra Pradesh,5,Krishna,1,"Krishna, Andhra Pradesh",16.31043,81.16987
2,Andhra Pradesh,5,Nellore,1,"Nellore, Andhra Pradesh",14.44606,79.98622
3,Andhra Pradesh,5,Prakasam,1,"Prakasam, Andhra Pradesh",15.63227,79.64718
4,Andhra Pradesh,5,Vizag,2,"Vizag, Andhra Pradesh",15.57429,80.12893


In [37]:
clean_data = clean_data[clean_data.Lat!='']

In [39]:
clean_data.to_csv("indian_covid_district.csv", index=False)

In [47]:
clean_data['No of Cases'] = clean_data['No of Cases'].astype(int)
clean_data['Lat'] = clean_data['Lat'].astype(float)
clean_data['Long'] = clean_data['Long'].astype(float)

In [61]:
# create map and display it
latitude = 23.26
longitude =  77.39
world_map = folium.Map(location=[latitude, longitude], zoom_start=4.5,tiles='Stamen Toner')

for lat, lon, Confirmed, state_name, district in zip(clean_data['Lat'], clean_data['Long'], clean_data['No of Cases'], clean_data['State'], clean_data['Name of District']):
    folium.CircleMarker([lat, lon],
                        radius=Confirmed,
                        popup = ('<strong>District_name</strong>: ' + str(district).capitalize() + '<br>'
                                 '<strong>State</strong>: ' + str(state_name) + '<br>'
                                '<strong>Confirmed Cases</strong>: ' + str(Confirmed) + '<br>'
                                ),
                        color='red',
                        
                        fill_color='red',
                        fill_opacity=0.7 ).add_to(world_map)
world_map

In [63]:
world_map.save("district.pdf")